In [48]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, Sequence, Annotated, Literal
from langchain_openai import AzureChatOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langgraph.graph import add_messages
from langgraph.types import Command
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import BaseMessage,HumanMessage,AIMessage,SystemMessage
from dotenv import load_dotenv
import os

In [49]:
load_dotenv()

True

LLM AND EMBEDDINGS MODELS

In [50]:
llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_LLM_DEPLOYMENT"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    )

In [51]:
embeddings = AzureOpenAIEmbeddings(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"), 
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

STATE OF THE GRAPH

In [52]:
class State(TypedDict):
    messages: Annotated[Sequence[BaseMessage],add_messages]
    email : Annotated[str, "Email address mentioned in the content"]
    job_role : Annotated[str, "Role mentioned in the content"]
    job_description : Annotated[str, "Job description mentioned in the content"]
    candidate_summary: Annotated[str, "Concise summary of candidate's relevant experience for the role"]
    final_email_content: Annotated[str, "The complete, drafted email for the recruiter"]

EXTRACTOR AGENT'S CODE

In [53]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Format(BaseModel):  # Class names should use PascalCase
    Email: str | None = Field(description="email provided in the user input")
    Role: int | None = Field(description="Role/designation of provided in the user input")
    Job_Description: str | None = Field(description="Job description provided in the user input")

In [ ]:
extractor_prompt = PromptTemplate(
    input_variables=["content"],
    template="""You are an expert in extracting information from text.
    Extract the email address, role, and job description from the following content:

    {content}

    Provide the extracted information in the following format:
    **Email**: <email>
    **Role**: <role>
    **Job Description**: <job_description>""",
)

exactor_llm = llm.with_structured_output(Format)

extractor_chain = extractor_prompt | exactor_llm

def extractor_agent(state: State) -> Command[Literal["rag_agent"]]:
    """An agent that extracts email, role and job description from the content provided by the user in order to send to rag agent which extracts relevant information from the resume
    """
    messages = state.get("messages", [])
    if not messages: #edge case to handle empty messages
        print("No messages found in state.")
        return "error_or_exit"
    last_message = messages[-1].content
    response = extractor_chain.invoke({"content": last_message})
    return Command(
        "rag_agent",
        update={
            "email": response.get("Email", ""),
            "job_role": response.get("Role", ""),
            "job_description": response.get("Job_Description", ""),
            "messages":[AIMessage(response)],
        },
    )

RAG AGENT CODE

In [55]:
from langchain_chroma import Chroma
from langchain_core.tools import tool
from pathlib import Path
from utils import get_embeddings

In [59]:
# get_embeddings(Path("artifacts/pdfs/RahulAGowda's Resume.pdf", embeddings=embeddings))

vectorstore = Chroma(
    collection_name="resume",
    embedding_function=embeddings,
    persist_directory="artifacts/embeddings/20250613_124237",
)
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 20}
)

In [60]:
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class Format(BaseModel):  # Class names should use PascalCase
    Email: str | None = Field(description="email provided in the user input")

In [61]:
@tool
def retriever_tool(query: str) -> str:
    """
    This tool searches and returns the information from the Stock Market Performance 2024 document.
    """

    docs = retriever.invoke(query)

    if not docs:
        return "I found no relevant information in the Stock Market Performance 2024 document."
    
    results = []
    for i, doc in enumerate(docs):
        results.append(f"Document {i+1}:\n{doc.page_content}")
    
    return "\n\n".join(results)


tools = [retriever_tool]

RAG_llm = llm.bind_tools(tools)

In [63]:
RAG_prompt = PromptTemplate(
    input_variables=["content"],
    template=
    """
    You are an expert career advisor and content synthesizer. Your task is to analyze the provide job role and job description and synthesize relevant information from the provided resume content to create a tailored response.
    
    Use the following job role and job description to guide your synthesis:
    **Job Role**: {job_role}
    **Job Description**: {job_description}
    
    **output format**:
    **candidate_summary**: <summary of candidate's relevant experience for the role>
    """
)

RAG_chain = RAG_prompt | RAG_llm

def rag_agent(state: State) -> Command[Literal["Mailer"]]:
    messages_content = state.get("messages", [])
    job_role = messages_content[0] if len(messages_content) > 0 else ""
    job_description = messages_content[1] if len(messages_content) > 1 else ""
    if not job_role and not job_description:  # edge case to handle empty messages
        print("No messages found in state.")
        return "error_or_exit"
    response = RAG_chain.invoke({"job_role": job_role, "job_description": job_description})
    return Command(
        "Mailer",
        update = {
            "candidate_summary": response.content,
            "messages": [AIMessage(response)],
        },
    )

MAILER AGENT CODE

In [ ]:
from langchain_google_community.gmail.utils import (
    build_resource_service,
    get_gmail_credentials,
)
from langgraph.prebuilt import create_react_agent
from langchain_google_community import GmailToolkit

token = os.getenv("TOKEN")
client_secrets_file = os.getenv("CLIENT_SECRET_FILE_PATH")
credentials = get_gmail_credentials(
    token_file=token,
    scopes=["https://mail.google.com/"],
    client_secrets_file=client_secrets_file,
)

api_resource = build_resource_service(credentials=credentials)
toolkit = GmailToolkit(api_resource=api_resource)
tools = toolkit.get_tools()

In [ ]:
mail_prompt = PromptTemplate(
    input_variables=["candidate_summary", "target_role", "company_name"],
    template="""
    You're an expert email writer. Your job is to draft a professional and concise email from a job candidate to a recruiter.

    **Goal:** Highlight the candidate's suitability for the **{target_role}** at **{company_name}** using the provided summary.

    **Information to use:**
    - **Candidate Summary:** {candidate_summary} (This is the core content for the email body)
    - **Target Role:** {target_role}
    - **Company Name:** {company_name} (If not specific, use "your company" or "your team")

    **Email Guidelines:**
    - **Subject:** Clear and professional (e.g., "Application for [Target Role]").
    - **Greeting:** Polite and standard (e.g., "Dear Hiring Team,").
    - **Body:** Briefly state interest, then integrate the `candidate_summary` to show a strong match with the role. Be direct and impactful.
    - **Closing:** Professional call to action for an interview, followed by a standard closing.
    - **Signature:** Use "Candidate Name" as a placeholder.
    - **Tone:** Confident, professional, and to the point.

    **Draft the complete email. Do NOT include any extra text before or after the email.**
    """
)

llm = llm.bind_tools(tools)

mail_chain = mail_prompt | llm

def mail_agent(state: State) -> Literal["END"]:
    "An agent that composes the final professional email to recruiters."

    # Retrieve all necessary information from the state
    candidate_summary = state.get("candidate_summary", "")
    candidate_email = state.get("email", "") # Using 'email' from state as candidate_email
    target_role = state.get("role", "")
    job_description = state.get("job_description", "")
    company_name = state.get("company_name", "your team") # Use a default if not found

    if not all([candidate_summary, candidate_email, target_role, job_description]):
        print("Missing essential information to compose email (summary, email, role, or job description).")
        return "error_node"
    final_email_content = mail_chain.invoke({
        "candidate_summary": candidate_summary,
        "candidate_email": candidate_email,
        "target_role": target_role,
        "job_description": job_description,
        "company_name": company_name
    })

    print(f"Generated Email Subject: (See email content for subject)")
    print(f"Generated Email Content: \n{final_email_content}")

    state["final_email_content"] = final_email_content

    return "END"